# Convert Commonwealth data into the flat table
------
### Start with table LogSheetEvent and pull information from other tables:
* Pull TripNo and ShotNo infos from file "Cwth foreign squid fishing data - constructed shot-by-shot";
* Pull Species info from CAAB code database;


In [12]:
import os
from datetime import date
import numpy as np
import pandas as pd

## Load Commonwealth data

In [13]:
data_dir = 'data_cw/' # Commonwealth data are stored in this folder

In [32]:
Species.columns

Index(['CAAB', 'COMMON_NAME'], dtype='object')

In [5]:
CwthData = pd.read_excel(data_dir + 'FTR_Operation_Catch.xlsx', index_col=None)

# do some preprocessing
CwthData = CwthData.fillna(0)

# load caab code and species name
Species = pd.read_excel(data_dir + 'CAAB-Name.xlsx', converters={'CAAB':int,'COMMON_NAME':str})
# remove empty COMMON_NAME species
Species = Species[Species['COMMON_NAME'].notnull()]

# link CAAB code to CommonName
CwthData = pd.merge(CwthData, Species, on='CAAB')

CwthData = CwthData.rename(columns={'FISHERY ID': 'FisheryID',
                                   'FISHING_TIME': 'EffortQty',
                                   'CATCH_WT': 'CatchWeight',
                                   'COMMON_NAME': 'SpeciesName',
                                   'GEAR': 'FishingMethodName',
                                   'OPERATION_NO': 'EndorsementCode'})


CwthData['LATITUDE'] = (CwthData['START_LATITUDE'] + CwthData['END_LATITUDE'])/2
CwthData['LONGITUDE'] = (CwthData['START_LONGITUDE'] + CwthData['END_LONGITUDE'])/2
CwthData = CwthData.drop(['START_LATITUDE', 'END_LATITUDE', 'START_LONGITUDE', 'END_LONGITUDE'], axis=1)

CwthData['CPUE'] = (CwthData['CatchWeight']/CwthData['EffortQty']).replace(np.inf, 0)

# round and fillnan
CwthData = CwthData.round({'LATITUDE': 2, 'LONGITUDE': 2, 'CPUE': 2})

CwthData = CwthData.drop(['POSITION_PRECISION',
                          'START_TIME',
                          'END_TIME',
                          'SEA_SURFACE_TEMP', 
                          'SPECIES_GRADE', 
                          'BOTTOM_DEPTH_MIN',
                          'BOTTOM_DEPTH_MAX'], axis=1)

# # remove species that has less than 100 records
# CwthData = CwthData.groupby('CAAB').filter(lambda x : len(x)>=1000)


In [50]:
CwthData = CwthData.round({'LATITUDE': 2, 'LONGITUDE': 2, 'CPUE': 2})

In [51]:
CwthData.head()

,FISHERY_ID,LOG_TYPE,BOAT_ID,FISHING_DATE,OPERATION_NO,GEAR,START_TIME,END_TIME,EffortQty,FISHING_DEPTH,CAAB,CatchWeight,COMMON_NAME,LATITUDE,LONGITUDE,CPUE
0,FTR,MB01,1062,1980-06-15,1,PTB,1980-06-15 07:05:00,1980-06-15 09:05:00,2.00,76.0,37331000,150.0,tilefishes,-18.93,119.56,75.00
1,FTR,MB01,1062,1980-06-30,3,PTB,1980-06-30 04:40:00,1980-06-30 07:00:00,2.33,73.0,37331000,30.0,tilefishes,-19.47,118.11,12.88
2,FTR,MB01,1062,1980-06-15,1,PTB,1980-06-15 07:05:00,1980-06-15 09:05:00,2.00,76.0,37335000,120.0,black kingfishes,-18.93,119.56,60.00
3,FTR,MB01,1062,1980-06-15,1,PTB,1980-06-15 07:05:00,1980-06-15 09:05:00,2.00,76.0,37335000,270.0,black kingfishes,-18.93,119.56,135.00
4,FTR,MB01,1062,1980-06-15,2,PTB,1980-06-15 10:25:00,1980-06-15 12:25:00,2.00,78.0,37335000,60.0,black kingfishes,-18.99,119.66,30.00


## Convert data to New South Wales Format

In [7]:
FishingActivityFlatTable = pd.DataFrame.from_dict(
        dict({'LogSheetID': [], 'LogSheetNo': [], 'LogSheetDate': [], 'LogSheetEventID': [], 'LogSheetEventDate': [],
             'FisheryID': [], 'FisheryName': [], 'EndorsementCode': [], 'FishingMethodCode': [],
             'FishingMethodName': [], 'UnitOfMeasureID': [], 'UnitOfMeasureName': [], 'FishingBusinessID': [],
             'FishingBusinessOwnerID': [], 'AuthorisedFisherID': [], 'SpeciesCode': [], 'SpeciesName': [],
             'CatchWeight': [], 'EffortQty': [], 'CPUE': [], 'Latitude': [], 'Longitude': [], 'Depth': [],
             'EstuaryCode': [], 'GridCode': [], 'SiteCode': [], 'SiteLatitude': [], 'SiteLongitude': [], 'GeneralEnabled': [], 'AbaloneEnabled': [],
              'LobsterEnabled': [], 'SutsEnabled': [], 'Jurisdiction': []
             })
    )

# LogsheetID,LogsheetNo,
# LogSheetDate,LogSheetEventID,LogSheetEventDate,FisheryID,FisheryName,EndorsementCode,FishingMethodCode,
# FishingMethodName,MeasureUnitID,MeasureUnitName,FishingBusinessID,FishingBusinessOwnerID,AuthorisedFisherID,
# SpeciesCode/CAABCode,SpeciesName,CatchWeight,EffortQty,CPUE,Latitude,Longitude,Depth,EstuaryCode,GridCode,
# SiteCode,GeneralEnabled,AbaloneEnabled,LobsterEnabled,SutsEnabled,Jurisdiction

In [8]:
CwthData.columns

Index(['FISHERY_ID', 'LOG_TYPE', 'BOAT_ID', 'FISHING_DATE', 'OPERATION_NO',
       'GEAR', 'START_LATITUDE', 'START_LONGITUDE', 'END_LATITUDE',
       'END_LONGITUDE', 'POSITION_PRECISION', 'START_TIME', 'END_TIME',
       'FISHING_TIME', 'BOTTOM_DEPTH_MIN', 'BOTTOM_DEPTH_MAX', 'FISHING_DEPTH',
       'SEA_SURFACE_TEMP', 'CAAB', 'SPECIES_GRADE', 'CATCH_WT'],
      dtype='object')

In [9]:
CwthData.head()

,FISHERY_ID,LOG_TYPE,BOAT_ID,FISHING_DATE,OPERATION_NO,GEAR,START_LATITUDE,START_LONGITUDE,END_LATITUDE,END_LONGITUDE,...,START_TIME,END_TIME,FISHING_TIME,BOTTOM_DEPTH_MIN,BOTTOM_DEPTH_MAX,FISHING_DEPTH,SEA_SURFACE_TEMP,CAAB,SPECIES_GRADE,CATCH_WT
0,FTR,MB01,1062,1980-06-15,1,PTB,-18.917,119.5,-18.95,119.617,...,1980-06-15 07:05:00,1980-06-15 09:05:00,2.0,74.0,82.0,76.0,NaN,37999999,TOTAL,30.0
1,FTR,MB01,1062,1980-06-15,1,PTB,-18.917,119.5,-18.95,119.617,...,1980-06-15 07:05:00,1980-06-15 09:05:00,2.0,74.0,82.0,76.0,NaN,37331000,TOTAL,150.0
2,FTR,MB01,1062,1980-06-15,1,PTB,-18.917,119.5,-18.95,119.617,...,1980-06-15 07:05:00,1980-06-15 09:05:00,2.0,74.0,82.0,76.0,NaN,37335000,POS 15 C X,120.0
3,FTR,MB01,1062,1980-06-15,1,PTB,-18.917,119.5,-18.95,119.617,...,1980-06-15 07:05:00,1980-06-15 09:05:00,2.0,74.0,82.0,76.0,NaN,23607000,TOTAL,30.0
4,FTR,MB01,1062,1980-06-15,1,PTB,-18.917,119.5,-18.95,119.617,...,1980-06-15 07:05:00,1980-06-15 09:05:00,2.0,74.0,82.0,76.0,NaN,37321000,TOTAL,180.0


In [10]:
# truncate the number to the number of limited decimals. for example: 12.679 -> 12.6
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

In [11]:
# this may takes 1 minutes per 10k records
MissLink_Species = 0
NAN_Lat_Lon = 0
count = 0
for index, row in CwthData.iterrows():
    if index % 1000 == 0:
        print("-- %s records processed ..." % index)
#     LogSheetEventID = row['SHOT_NO'] # everything is linked by this ID
    LogSheetEventID = index
    LogSheetEventDate = row['FISHING_DATE']
#     LogSheetID = row['TRIP_NO']
    LogSheetID = index
    
    # convert bussiness, fisher info
    LogSheetNo = LogSheetID
    LogSheetDate = LogSheetEventDate
    FishingBusinessID = "NA"
    FishingBusinessOwnerID = "NA"
    AuthorisedFisherID = "NA"
    EndorsementCode = row['OPERATION_NO']
    
    # convert fishing method
    FishingMethodCode = str(row['GEAR']).strip()
    FishingMethodName = FishingMethodCode
    UnitOfMeasureID = -1
    UnitOfMeasureName = 'FishingTime'
    
    # convert fishery infos
    FisheryID = row['FISHERY_ID']
    FisheryName = row['FISHERY_ID']
    
    # convert catchweight
    CatchWeight = np.nan_to_num(row['CATCH_WT'])
    
    # convert shot duration
#     shot_start_time = row['FISHING_START_TIME']
#     shot_end_time = row['FISHING_END_TIME']
#     shot_duration = shot_end_time - shot_start_time
    EffortQty = round(row['FISHING_TIME'], 2)
#     print(shot_start_time, shot_end_time, shot_duration.seconds//60)
#     EffortQty = shot_duration.seconds//60 # get the minutes
    if EffortQty == 0:
        CPUE = 0
    else:
        CPUE = round(CatchWeight/EffortQty, 2)
    
    # convert species info
    SpeciesCode = row['CAAB']
    if SpeciesCode not in D_Species:
        SpeciesName = 'Unknown'
        MissLink_Species += 1
    else:
        SpeciesName = D_Species[SpeciesCode]
    
    # convert coordinates data from LogSheetEventAreas table
    latitude_start = row['START_LATITUDE']
    longitude_start = row['START_LONGITUDE']
    latitude_end = row['END_LATITUDE']
    longitude_end = row['END_LONGITUDE']
    Latitude = (latitude_end - latitude_start)/2 + latitude_start
    Longitude = (longitude_end - longitude_start)/2 + longitude_start
#     print(Latitude, Longitude, np.isnan(Latitude), np.isnan(Longitude))
    if np.isnan(Latitude) or np.isnan(Longitude):
        NAN_Lat_Lon += 1
        continue
    ## calculate the site information, coordinates within 0.1 degree will belong to the samve site, 
    ## for example, Lat=123.22, Lon=-45.36 belong to site: (123.2, -45.3)
    SiteLatitude = truncate(Latitude, decimals=1) # the tenth degree is taken to compute site
    SiteLongitude = truncate(Longitude, decimals=1)
    
    # convert depth info
#     depth_min = row['BOTTOM_DEPTH_MIN']
#     depth_max = row['BOTTOM_DEPTH_MAX']
#     Depth = round((depth_min + depth_max)/2)
#     print(type(depth_start))
    Depth = row['FISHING_DEPTH']
#     print(depth_start, depth_end, Depth)
    
    # convert grid/site infor, which currently is missing, set to some defaults
    GridCode = 'NA'
    SiteCode = 'NA'
    EstuaryCode = 'NA'
    GeneralEnabled = 1
    AbaloneEnabled = 1
    LobsterEnabled = 1
    SutsEnabled = 1
    Jurisdiction = 'Commonwealth'
    
    # insert into the flat table
    data = pd.DataFrame(
                dict({'LogSheetID': LogSheetID, 'LogSheetNo': LogSheetNo, 'LogSheetDate': LogSheetDate, 
                      'LogSheetEventID': LogSheetEventID, 'LogSheetEventDate': LogSheetEventDate,
                      'FisheryID': FisheryID, 'FisheryName': FisheryName, 'EndorsementCode': EndorsementCode, 
                      'FishingMethodCode': FishingMethodCode, 'FishingMethodName': FishingMethodName, 
                      'UnitOfMeasureID': UnitOfMeasureID, 'UnitOfMeasureName': UnitOfMeasureName, 
                      'FishingBusinessID': FishingBusinessID, 'FishingBusinessOwnerID': FishingBusinessOwnerID, 
                      'AuthorisedFisherID': AuthorisedFisherID, 'SpeciesCode': SpeciesCode, 
                      'SpeciesName': SpeciesName, 'CatchWeight': CatchWeight, 'EffortQty': EffortQty, 'CPUE': CPUE,
                      'Latitude': Latitude, 'Longitude': Longitude, 'Depth': Depth, 'EstuaryCode': EstuaryCode,
                      'GridCode': GridCode, 'SiteCode': SiteCode, 'SiteLatitude': SiteLatitude, 'SiteLongitude': SiteLongitude, 
                      'GeneralEnabled': GeneralEnabled, 'AbaloneEnabled': AbaloneEnabled, 'LobsterEnabled': LobsterEnabled, 'SutsEnabled': SutsEnabled,
                      'Jurisdiction': Jurisdiction}), index=[index]
    )
    
    FishingActivityFlatTable = FishingActivityFlatTable.append(data)
    
print('# %s, species cannot find in CAAB database' % MissLink_Species)
print('# %s, Latitude or Longitude is missing' % NAN_Lat_Lon)
# print('# %s, miss link between LogSheetEvent and LogSheet' % MissLink_LogSheet)
# print('# %s, miss link between LogSheetEvent and LogSheetEventCatch' % MissLink_LogSheetEventCatch)
# print('# %s, miss link between LogSheetEvent and LogSheetEventArea' % MissLink_LogSheetEventArea)
    
    

-- 0 records processed ...
-- 1000 records processed ...
-- 2000 records processed ...
-- 3000 records processed ...
-- 4000 records processed ...
-- 5000 records processed ...
-- 6000 records processed ...
-- 7000 records processed ...
-- 8000 records processed ...
-- 9000 records processed ...
-- 10000 records processed ...
-- 11000 records processed ...
-- 12000 records processed ...
-- 13000 records processed ...
-- 14000 records processed ...
-- 15000 records processed ...
-- 16000 records processed ...
-- 17000 records processed ...
-- 18000 records processed ...
-- 19000 records processed ...
-- 20000 records processed ...
-- 21000 records processed ...
-- 22000 records processed ...
-- 23000 records processed ...
-- 24000 records processed ...
-- 25000 records processed ...
-- 26000 records processed ...
-- 27000 records processed ...
-- 28000 records processed ...
-- 29000 records processed ...
-- 30000 records processed ...
-- 31000 records processed ...
-- 32000 records proc

-- 260000 records processed ...
-- 261000 records processed ...
-- 262000 records processed ...
-- 263000 records processed ...
-- 264000 records processed ...
-- 265000 records processed ...
-- 266000 records processed ...
-- 267000 records processed ...
-- 268000 records processed ...
-- 269000 records processed ...
-- 270000 records processed ...
-- 271000 records processed ...
-- 272000 records processed ...
-- 273000 records processed ...
-- 274000 records processed ...
-- 275000 records processed ...
-- 276000 records processed ...
-- 277000 records processed ...
-- 278000 records processed ...
-- 279000 records processed ...
-- 280000 records processed ...
-- 281000 records processed ...
-- 282000 records processed ...
-- 283000 records processed ...
-- 284000 records processed ...
-- 285000 records processed ...
-- 286000 records processed ...
-- 287000 records processed ...
-- 288000 records processed ...
-- 289000 records processed ...
-- 290000 records processed ...
-- 29100

-- 517000 records processed ...
-- 518000 records processed ...
-- 519000 records processed ...
-- 520000 records processed ...
-- 521000 records processed ...
-- 522000 records processed ...
-- 523000 records processed ...
-- 524000 records processed ...
-- 525000 records processed ...
-- 526000 records processed ...
-- 527000 records processed ...
-- 528000 records processed ...
-- 529000 records processed ...
-- 530000 records processed ...
-- 531000 records processed ...
-- 532000 records processed ...
-- 533000 records processed ...
-- 534000 records processed ...
-- 535000 records processed ...
-- 536000 records processed ...
-- 537000 records processed ...
-- 538000 records processed ...
-- 539000 records processed ...
-- 540000 records processed ...
-- 541000 records processed ...
-- 542000 records processed ...
-- 543000 records processed ...
-- 544000 records processed ...
-- 545000 records processed ...
-- 546000 records processed ...
-- 547000 records processed ...
-- 54800

KeyboardInterrupt: 

In [ ]:
# save the converted FlatTable to local files for further analysis
FishingActivityFlatTable.to_pickle(data_dir + 'FishingActivityFlatTable.pkl')

In [ ]:
# FishingActivityFlatTable['LogSheetEventDate'][0]<2010

In [ ]:
FishingActivityFlatTable.head()